In [1]:
import numpy as np
import pandas as pd
from operator import itemgetter
import re
import random

In [2]:
# To always get the same random results
random.seed(777)

In [3]:
# Load the original dataset and delete null values of price, sellers, stock and category
dataset = pd.read_csv(filepath_or_buffer='originalDataset.csv', encoding = 'utf8')
dataset = dataset[dataset['price'].notna()]
dataset = dataset[dataset['sellers'].notna()]
dataset = dataset[dataset['number_available_in_stock'].notna()]
dataset = dataset[dataset['amazon_category_and_sub_category'].notna()]

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4286 entries, 0 to 9998
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   uniq_id                                      4286 non-null   object 
 1   product_name                                 4286 non-null   object 
 2   manufacturer                                 4284 non-null   object 
 3   price                                        4286 non-null   object 
 4   number_available_in_stock                    4286 non-null   object 
 5   number_of_reviews                            4276 non-null   object 
 6   number_of_answered_questions                 3995 non-null   float64
 7   average_review_rating                        4276 non-null   object 
 8   amazon_category_and_sub_category             4286 non-null   object 
 9   customers_who_bought_this_item_also_bought   3959 non-null   object 
 10  

In [5]:
# dataset.head()

In [6]:
#dataset.tail()

In [7]:
attributes = ['product_name','price','number_available_in_stock','amazon_category_and_sub_category','sellers']
# dataset[attributes]

In [8]:
# Removing non-printable characteres
for att in attributes:
    dataset[att] = dataset[att].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))

In [9]:
# ========================================= PREPROCESSING DATASET ==============================================
def removePatterns(str, patterns):
    for pattern in patterns:
        str = re.sub(pattern, '', str)
    return str

def getPrice(str):
    str = str[1:]
    str = str.replace(',','')
    return float(str)

def getStock(str):
    return int(str.split()[0])

def getCatAndSub(str):
    idxSubCat = str.find(' > ')
    category = str[:idxSubCat]
    subcategory = str[idxSubCat+3:]
    return category, subcategory
    
    
def getSellers(str):
    # List Initialization
    sellersList = []
    idxList = str.find('[')
    # If there is no array, but one seller only
    if idxList < 0:
        idxList = 10
    # Example: "Seller_name_1"=>"{Amazon.co.uk", "Seller_price_1"=>"£3.42}, {...}, ..., {...}"
    sellersStrList = str[idxList+2:-3].split('}, {')
    for sellerAndPriceStr in sellersStrList:
        # Example: "Seller_name_1"=>"Amazon.co.uk" 
        sellerStr = sellerAndPriceStr.split(', "Seller_price')[0]
        # Example: "Seller_price_1"=>"£3.42"
        priceStr = sellerAndPriceStr.split(', "Seller_price')[1]
        # Example: Amazon.co.uk
        idxAssign = sellerStr.find('=>')
        seller = sellerStr[idxAssign+3:-1]
        strRemovals = ["'", '"', ';', '\(' , '\)']
        seller = removePatterns(seller, strRemovals)
        # Example: £3.42
        idxAssign = priceStr.find('=>')
        price = getPrice(priceStr[idxAssign+3:-1])
        sellersList.append((seller, price))
    return sellersList

def orderBy(dictionary):
    return sorted(dictionary, key=itemgetter('seller', 'category', 'product', 'stock', 'price'))

def groupBy(dictionary, order):
    # Initialize previous tuple
    prev = dictionary[0]
    # For each row in the dictionary
    for row in dictionary:
        # Initialize current row as in the same group as previous
        sameGroup = True
        # Compare each current att to the previous one (for each att in the group criteria)
        for att in order:
            if row[att] != prev[att]: 
                sameGroup = False;
                break;
        # Update the group id
        if sameGroup:
            row["idGroup"] = prev["idGroup"]
        else:
            row["idGroup"] = prev["idGroup"] + 1
        # The current tuple becomes the previous
        prev = row
            

def storeTuple(dictionary, idGroup, product, stock, category, sellersAndPrices):
    # EACH INGREDIENT GENERATES A NEW TUPLE
    for sellerAndPrice in sellersAndPrices:        
        seller = sellerAndPrice[0]
        price  = sellerAndPrice[1]
        # random stock with maximum value the original stock
        randStock = int(random.random()*stock + 1)
        
        tuple = {'idGroup':idGroup, 'seller':seller, 'category':category, 
                 'product':product, 'stock':randStock, 'price':price}
        dictionary.append(tuple)

strRemovals = ["'", '"', ';']
dictionary = []
idGroup = 0
for tuple in dataset.iterrows():
    product = removePatterns(tuple[1]['product_name'], strRemovals)
    stockStr = removePatterns(tuple[1]['number_available_in_stock'], strRemovals)
    stock = getStock(stockStr)
    catAndSub = removePatterns(tuple[1]['amazon_category_and_sub_category'], strRemovals)
#     category, subcategory = getCatAndSub(catAndSub)
    sellersStr = tuple[1]['sellers']
    sellersAndPrices = getSellers(sellersStr)
    storeTuple(dictionary, idGroup, product, stock, catAndSub, sellersAndPrices)
#     idGroup += 1
    
#order and group dictionary
dictionary = orderBy(dictionary)
groupBy(dictionary, ['seller'])

# new data frame
newDataset = pd.DataFrame(dictionary)    

In [10]:
# newDataset.info()
newDataset.head()

idGroup              seller  \
0        0   Peachy Wing JAPAN   
1        1         #1 FOR TOYS   
2        1         #1 FOR TOYS   
3        1         #1 FOR TOYS   
4        1         #1 FOR TOYS   

                                            category  \
0                   Characters & Brands > Action Man   
1                Characters & Brands > Disney > Toys   
2  Die-Cast & Toy Vehicles > Toy Vehicles & Acces...   
3     Figures & Playsets > Science Fiction & Fantasy   
4     Figures & Playsets > Science Fiction & Fantasy   

                                             product  stock  price  
0  Bandai Tamashii Nations Shenlong x Lingyin Hua...     14  57.20  
1  Hasbro A7902 Guardians of The Galaxy Big Blast...      6  15.95  
2    Hot Wheels Batman 2015: Batman Begins Batmobile      4   7.25  
3                                 Moshi Rox Tin Game      2   9.25  
4  Mr. Bean British Comedian Bendable Poseable Ac...     10   8.75

In [11]:
# ========================================= STORING DATASET ==============================================
# ======== MY RELATIONAL CONDITION FORMAT
def getHeader(row, sizes, columns=[]):
    header = ""
    if(len(columns) == 0): header = "id(float,1);"
    currSize = 0
    currType = ""
    i = 0
    for title in row:
        # Verify the column is in the list
        if len(columns) == 0 or title in columns:
            if isinstance(row[title], str):
                currType = "string"
                currSize = sizes[i] + 1
            else:
                currType = "float"
                currSize = 1
            # Add element to header
            header += title + "(" + currType + "," + str(currSize) + ");"
        # Increment
        i += 1
    # printing the header
    return header

def getFormat(value):
    if isinstance(value, str):
        return '"' + str(value) + '"; '
    else:
        return str(value) + "; "
    

def saveData(dictionary, sizes):
    f = open("../T1.data", "w")
    f.write(getHeader(dictionary[0], sizes)+"\n")
    for i in range(len(dictionary)):
        # Id
        f.write(str(i)+"; ")
        for value in dictionary[i].items():
            f.write(getFormat(value[1]))
        f.write("\n")
    f.close()
    
def saveGroups(dictionary):
    f = open("../TG.data", "w")
    for idRow in range(len(dictionary)):
        f.write(str(idRow) + "\t" + str(dictionary[idRow]["idGroup"]) + "\n")
    f.close()

In [12]:
# ============================ CREATION OF THE TESTS ===================================
def readPacks(source):
    packs = []
    lastIdPack = -1
    f = open(source);
    for line in f:
        idPack, idProd = list(map(int,line.split()))
        if idPack != lastIdPack:
            lastIdPack = idPack
            packs.append([])
        packs[idPack].append(idProd)
    f.close();
    return packs

def createTests(dictionary, sizes, columns, packs):
    # For each case from 2 to 100
    for iTest in range(2,101):
        # Open the folder of current test case
        f = open("../TESTS/TEST "+str(iTest)+ "/T2.data", "w")
        # Store the header
        f.write(getHeader(dictionary[0], sizes, columns) + "\n")
        # Peek a pack
        idPack = int(random.random()*len(packs))
        # Variable number of requirements. x between 2-10 or lengh of path if lower than x
        sizeT2 = min(int(random.random()*9 + 2), len(packs[idPack])-1)
        # Generate all the requirements
        iTuples = random.sample(packs[idPack], sizeT2)
        for iTuple in iTuples:
            # Iterating columns
            for value in dictionary[iTuple].items():
                if value[0] in columns:
                    if value[0] == 'stock':
                        f.write(getFormat(int(value[1]*(0.2 + random.random()))))
                    elif value[0] == 'price':
                        f.write(getFormat(int(value[1]*(1.0 + random.random()))))
                    else:
                        f.write(getFormat(value[1]))
            # end tuple with end line
            f.write("\n")
        # Close current test file
        f.close()
    


In [13]:
# ========== SQL DATABASE FORMAT    

def getSQLCreateTable(row, tableName, sizes):
    header = "DROP TABLE IF EXISTS " + tableName + ";\n"
    header += "CREATE TABLE " + tableName + "(\n"
    header += "\tid INT PRIMARY KEY \n"
    currSize = 0
    currType = ""
    i = 0
    for title in row:
        # Separation between attributes
        header += "\t, "
        # Find the type of the attribute
        if isinstance(row[title], str):
            currType = "VARCHAR"
            currSize = sizes[i] + 1
            # Add element to header
            header += title + " " + currType + "(" + str(currSize) + ")\n"
        else:
            currType = "FLOAT"
            header += title + " " + currType + "\n"
        
        # Increment
        i += 1
    # closing the sentence
    header += ");\n"
    # printing the instruction
    return header

def getSQLFormat(value):
    if isinstance(value, str):
        return ", \"" + str(value) + "\""
    else:
        return ', ' + str(value)

def saveSQLData(dictionary, tableName, sizes, pgSize = 10000):
    f = open("AmazonToysDB.sql", "w")
    f.write(getSQLCreateTable(dictionary[0], tableName, sizes)+"\n")
    # Each tuple in the dataset
    for idRow in range(len(dictionary)):
        # Separation between tuples and pages
        if idRow % pgSize == 0:
            if idRow > 0: f.write(";\n")
            f.write("INSERT INTO " + tableName + " VALUES\n")
        else:
            f.write(", ")
        # Start writing the tuple
        f.write("(" + str(idRow)) # id
        # Each attribute of the tuple
        for value in dictionary[idRow].items():
            f.write(getSQLFormat(value[1]))
        f.write(")\n")
    # Closing last page
    if len(dictionary) % pgSize > 0:
        f.write(";\n")
    # Closing file
    f.close()
    



In [14]:
# Max length of strings in the new dataset
measurer = np.vectorize(len)
sizes = measurer(newDataset.values.astype(str)).max(axis=0)
# print(sizes)
saveData(dictionary, sizes)
saveGroups(dictionary)
saveSQLData(dictionary, "T1", sizes)

In [15]:
# Create tests 2 - 100
packs = readPacks('packs.csv')
createTests(dictionary, sizes, ['category', 'product', 'stock', 'price'], packs)